<br><br>
<span style="font-size:2em;font-weight:lighter;">194.025 Introduction to Machine Learning</span><br>
<span style="font-size:3em;font-weight:normal;line-height:70%;">Assignment 4: How to plant a Tree</span>

---



Welcome to the 4th assignment of our course **Introduction to Machine Learning**. You will be able to earn up to a total of 10 points. Please read all descriptions carefully to get a full picture of what you have to do. 

**Remark:** Some code cells are put to read-only. Please execute them regardless as they contain important code. You can run a jupyter cell by pressing `SHIFT + ENTER`, or by pressing the play button on top (in the row where you can find the save button). Cells where you have to implement code contain the comment `# YOUR CODE HERE` followed by `raise NotImplementedError`. Simply remove the `raise NotImplementedError`and insert your code.

Some other code cells start with the comment `# hidden tests ...`. Please do not change them in any way as they are used to grade the tasks after your submission.

This assignment will cover the implementation of simpler machine learning models. You will have to implement a very simple linear classifier as well as your very own decision tree.

#### Linear Classifier (3 Points)

Implement the linear classifier as discussed in the lecture "Basic Algorithms II".
Make sure that you use `la.pinv` for inverting your matrices. You could think about when this is necessary.

In [2]:
import numpy as np
import numpy.linalg as la

def linear_classifier_fit(X, y):
    """Train the simple linear classifier as described in the lecture, given data X and labels y.

    Input: 
        X: np float array of shape (n_samples, data_dimension) 
        y: np float array of shape (n_samples, ) values should be in {-1,1}

    Output:
        w: np float array of shape (data_dimension + 1, )
    """
    # YOUR CODE HERE

    # In the slides, an additional row with ones was added
    n_samples = X.shape[0]
    ones_column = np.ones((n_samples, 1))
    X_aug = np.hstack([ones_column, X])

    # Prepare it for la.pinv() function
    X_ = X_aug.T

    # Compute the weights
    w = la.pinv(X_ @ X_.T) @ X_ @ y

    return w

In [3]:
def linear_classifier_transform(X, w):
    """Compute predictions for data X given the weight vector w.

    Input: 
        X: np float array of shape (n_samples, data_dimension) 
        w: np float array of shape (data_dimension + 1, )   

    Output:
        p: np float array of shape (n_samples, ) values should be in {-1,1}

    """
    # YOUR CODE HERE

    b = w[0]
    a = w[1:]

    # Compute scores
    scores = X @ a + b

    # Predict -1 or 1
    p = np.sign(scores)

    return p

In [4]:
# hidden tests - DO NOT CHANGE THIS CELL

In [5]:
# hidden tests - DO NOT CHANGE THIS CELL

In [6]:
# hidden tests - DO NOT CHANGE THIS CELL

In [7]:
# hidden tests - DO NOT CHANGE THIS CELL

#### Train your model on a more complicated Data Set (2 Points)

The file `chipotle_stores.csv` contains information on stores of a large fast food chain in the US. 
The column `state` of the dataset contains the state of each store, `longitude` and `latitude` contain the position of each store. 

Load the dataset using some suitable means. Create a numpy array `X` of shape `(n_samples, 2)` that stores latitude and longitude of each store, as well as a vector `y` of shape `(n_samples,)` that contains a `1` if the store is in one of the states
```"Washington", "Oregon", "California", "Idaho", "Nevada", "Utah", "Arizona", "New Mexico", "Colorado", "Wyoming", "Montana"```
and `-1` otherwise.


In [8]:
import numpy as np
import pandas as pd

In [9]:
# store dataset in these variables
X = None
y = None

# YOUR CODE HERE

data = pd.read_csv("chipotle_stores.csv")

X = np.column_stack([data["latitude"].to_numpy(), data["longitude"].to_numpy()])
y = np.ones(X.shape[0])

state_col = ["Washington", "Oregon", "California", "Idaho", "Nevada", "Utah", "Arizona", "New Mexico", "Colorado", "Wyoming", "Montana"]

for i in range(y.shape[0]):
    # Check if the state corresponding to the row exists in state_col
    if data.iloc[i]["state"] not in state_col:
        y[i] = -1

In [10]:
# store predictions in this variable
predictions = None

# YOUR CODE HERE

w = linear_classifier_fit(X, y)
predictions = linear_classifier_transform(X, w)

In [11]:
# hidden tests - DO NOT CHANGE THIS CELL

In [12]:
# hidden tests - DO NOT CHANGE THIS CELL

#### Decision Tree Classifier (1.5 Points)

Implement the Gini index as defined in the lecture. 
To be able to work with more general data, we will implement it with array input that gives the outcome of any test that outputs boolean values.
That is, e.g., given some numerical attribute 
```
a = np.array([1,2,3,4,5,6,7,8])
y = np.array([0,1,0,0,1,1,1,1])
```

The input to the gini function may be
```x = (a > 4.5)```.
This expression (for numpy arrays!) produces the array
```[0,0,0,0,1,1,1,1]```.
We then want to compute 
`gini(x, y)`
which should produce the output `0.1875`.

In [13]:
import numpy as np
import numpy.linalg as la


def gini(x, y):
    """Compute the gini index of a split.

    x are the labels that your split produces.
    y are the binary labels.

    Input:
        x: np float array of shape (n_samples, ) values should be in {0,1}
        y: np float array of shape (n_samples, ) values should be in {0,1}
    Output:
        g_x: float value

    """

    # YOUR CODE HERE

    # Group indices
    group_0 = (x == 0)
    group_1 = (x == 1)

    n = len(y)

    g_X = 0.0

    for group in [group_0, group_1]:
        size = np.sum(group)
        if size == 0:
            continue  # avoid division by zero

        # Get class proportions
        p1 = np.sum(y[group] == 1) / size
        p0 = np.sum(y[group] == 0) / size

        # Gini for this group
        gini_group = 1 - (p1**2 + p0**2)

        # Weighted contribution to total Gini
        g_X += (size / n) * gini_group

    return g_X    

In [14]:
# hidden tests - DO NOT CHANGE THIS CELL

In [15]:
# hidden tests - DO NOT CHANGE THIS CELL

#### Decision Tree Classifier II (3.5 Points)

Using the gini function, you should now implement the top down induction of decision trees algorithm, as presented in the lecture.
Below, you find the dataset from the lecture to play around.

In [16]:
observations = np.array([
    [0,1,1,0],
    [0,0,0,1],
    [0,1,0,0],
    [0,0,0,0],
    [1,0,1,0],
    [1,0,1,1]
    ], dtype=bool)
drinking = np.array([1,1,0,1,0,0], dtype=bool)

Implement the two functions below. 

As a stopping criterion, use minimum leaf size, that is:
If the training data that you obtain in some test has at most `min_leaf_size` examples, create a leaf that predicts the majority class of that data.


You may represent your decision tree however you like, but one common way to do so is to create some objects that can store the relevant information for a decision tree node and its children, 
for example like this:
```
class DecisionTreeNode:
    isLeaf: bool = False
    splitIndex: int = -1
    prediction: bool = False
    evalsTrue = None
    evalsFalse = None
```
Here, `evalsTrue` and `evalsFalse` will again be `DecisionTreeNode` objects storing information of the tests/leaf nodes that follow the current test.



In [22]:
# You can use this class to represent your tree (as described above) or overwrite it with any other method of your choice


class DecisionTreeNode:
    # YOUR CODE HERE
    #raise NotImplementedError()
    def __init__(self, isLeaf, splitIndex: int = -1, prediction: bool = False, evalsTrue=None, evalsFalse=None):
        self.isLeaf = (evalsTrue == None and evalsFalse == None)
        self.splitIndex = splitIndex
        self.prediction = prediction
        self.evalsTrue = evalsTrue
        self.evalsFalse = evalsFalse

    def setEvalsTrue(self, evalsTrue):
        self.evalsTrue = evalsTrue

    def setEvalsFalse(self, evalsFalse):
        self.evalsFalse = evalsFalse

    def getSplitIndex(self):
        return self.splitIndex

In [23]:
def decision_tree_fit(X, y, min_leaf_size=1):
    """Fit the decision tree.

    Parameters
    ----------
    X : array_like of shape (n_samples, data_dimension)
        Training instances.
    y : array_like of shape (n_samples)
        Training labels.
    min_leaf_size : int, default=1
        Minimum leaf size.

    Returns
    -------
    tree : DecisionTreeNode
        The fitted decision tree.
    """
    # YOUR CODE HERE
    #raise NotImplementedError()

    if len(X) <= min_leaf_size:
        # Compute majority class
        class_counts = np.bincount(y)
        majority_class_idx = np.argmax(class_counts)
        return DecisionTreeNode(isLeaf=True, prediction=majority_class_idx)

    n_samples, feature_size = X.shape

    # Compute Gini index for each feature
    gini_values = [gini(X[:, i], y) for i in range(feature_size)]

    # Find the feature with the lowest Gini index
    gini_idx = np.argmin(gini_values)

    # Split data
    mask_true = X[:, gini_idx] == 1
    mask_false = ~mask_true  # X[:, gini_idx] == 0

    X_true, y_true = X[mask_true], y[mask_true]
    X_false, y_false = X[mask_false], y[mask_false]

    # Recursively build subtrees
    child_true = decision_tree_fit(X_true, y_true, min_leaf_size)
    child_false = decision_tree_fit(X_false, y_false, min_leaf_size)

    # Return root node
    return DecisionTreeNode(
        isLeaf=False,
        splitIndex=gini_idx,
        prediction=None,         # not needed here; only for leaves
        evalsTrue=child_true,
        evalsFalse=child_false
    )

In [19]:
def decision_tree_transform(X, tree):
    """Compute the labels of the given instances predicted by the given decision tree.

    Parameters
    ----------
    X : array_like of shape (n_samples, data_dimension)
        Iinstances to be transformed.
    tree : DecisionTreeNode
        The decision tree which classifies the instances.

    Returns
    -------
    results : np.ndarray of shape (n_samples)
        The computed labels.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    results = []

    for x in X:
        node = tree
        while not node.isLeaf:
            if x[node.splitIndex]:
                node = node.evalsTrue
            else:
                node = node.evalsFalse
        results.append(node.prediction)

    results = np.array(results)
    
    return results

In [ ]:
# hidden tests - DO NOT CHANGE THIS CELL
# Test if your decision tree can memorize training data.

In [ ]:
# hidden tests - DO NOT CHANGE THIS CELL
# Test if your decision tree classifies everything correctly if x_i = y is one of the features.